# Links

This file creates a class 'Links', whose objects contain the essential parameters for a directionaal link like : nodes(source, destination pair), weight, availablity of various wavelengths, etc.

Furthermore, there are various methodsto initialize, update, displaying links. Also, methods to calculate Path resources, TUR, and FF approach are created.

In [6]:
class Links:

    ## Class variables
    
    #links = []
    #indices : a mask
    #unordered_indices = []    # Not useful in case of bidirectional graph
    #numLinks = len(ordered_indices)
    #key_reserve

    channel_ts = {1 : 8, 2 : 10, 3 : 12}
    n_ts = {"tdc" : 47, "high" : channel_ts[1], "medium" : channel_ts[2], "low" : channel_ts[3]}    # Creating a dictionary with sl and the corresponding number of time-slots in the channel

    channel = {"tdc" : 0, "high" : 1, "medium" : 2, "low" : 3}    # A dictionary for channel from sl 
    priority = {0 : "tdc", 1 : "high", 2 : "medium", 3 : "low"}    # A dictionary for sl from channel
    
    total_ts = channel_ts[1] + channel_ts[2] + channel_ts[3]    # Total ts in each link
    
    def __init__(self, nodes, weight):    # nodes = (s, d); is a tuple so as to not confuse s and d individually
        self.nodes = nodes
        self.weight = weight
        
        self.lambda_tdc = np.ones(47, dtype = bool)
        
        #self.total_ts = Links.total_ts
        
        self.occupied_ts = np.zeros(4).astype(int)    # Stores the number of occupied time slots [total, q1, q2, q3]
        self.available_ts = np.array([Links.total_ts,    # Stores the number of available time slots [total, q1, q2, q3]
                                     Links.channel_ts[1], Links.channel_ts[2], Links.channel_ts[3]])
        
        self.lambda_q1 = np.ones(Links.channel_ts[1], dtype = bool)    # for high sl
        self.lambda_q2 = np.ones(Links.channel_ts[2], dtype = bool)    # for medium sl    
        self.lambda_q3 = np.ones(Links.channel_ts[3], dtype = bool)    # for low sl


    def tdc(self, key, text):
        
        key_string = np.array2string(key, separator = "").lstrip('[').rstrip(']')
        dec = int(key_string, 2)
        hexadecimal = hex(dec)

        
    
    def bb84(self, cr):    
        # establish a secure key 
        sl = cr.sl
        key = ''
        
        %run ./iterated_bb84_3.ipynb    # Will establish a secure--error-corrected key of required length(sl)
        
        return key
        
        
        #flag = 1

        #while flag :
        #    %run ./bb84_qiskit.ipynb
        #    reserve += key

        #    try :
        #        allocated_key = reserve[:qkey[sl]]
            
        #    flag = bob_sample == alice_sample

    
   
    def update_link(self, channel, slot):
        #QSC = ["lambda_q1", "lambda_q2", "lambda_q3"]
        QSC = [1, 2, 3]
        TDC = [0]
        AC = QSC + TDC
        #if channel not in AC or ts < 0:
            #print("Invalid Update request!")

        if channel == 0:
            self.lambda_tdc[slot] = False
        
        elif channel == 1 and slot < len(self.lambda_q1):
            self.lambda_q1[slot] = False 
            
        elif channel == 2 and slot < len(self.lambda_q2):
            self.lambda_q2[slot] = False
            
        elif channel == 3 and slot < len(self.lambda_q3):
            self.lambda_q3[slot] = False

        else:
            print("Invalid time slot value")

        if channel != 0:
            self.occupied_ts[channel] += 1 
            self.occupied_ts[0] += 1

            self.available_ts[channel] -= 1 
            self.available_ts[0] -= 1
        
    
    def display_info(self, wl_info = False):
        q1_count = self.available_ts[1] # nonzero => Available slots
        q2_count = self.available_ts[2] #== True
        q3_count = self.available_ts[3] #== True
        
        tdc_count = np.count_nonzero(self.lambda_tdc) #== True
        print(f"Link {self.nodes} : lambda_tdc_count = {tdc_count}, lambda_q1_count = {q1_count}, lambda_q2_count = {q2_count}, lambda_q3_count = {q3_count}, occupied_ts = {self.occupied_ts}, available_ts = {self.available_ts}")

        if wl_info:    # To show the wavelength occupancy
            print(f"QSC: \n q1 : {self.lambda_q1}, q2 : {self.lambda_q2}, q3 : {self.lambda_q3}, tdc : {self.lambda_tdc}")
        
  ###################################################################################################################################################
    
    @classmethod
    def initialize_links(cls, edges):    # Initializing all the links and the individual link resources

        all_values = [nodes for row in edges for nodes in row[:2]]    # A list of all the values/nodes in first 2 columns : s & d
        m = min(all_values)
        M = max(all_values)
        numNodes = max(all_values) - min(all_values) + 1     # Correction for the convention in indexing of nodes in the data of edges

        links = np.zeros([M+1, M+1], dtype = object)    # Will have redundant entries. M is the highest node index

        ordered_indices = []
        for (s, d, w) in edges:
            nodes = (s, d)
            link = cls(nodes, w)
            links[s, d] = link   
            ordered_indices.append(nodes)
        # In the above call of __init__ constructor, the wavelength resources have also been initialized to all available(True)
                    
        indices = links != 0    # A mask that holds the location of each link. NOTE : It stores ordered pair

        cls.links = links    # Matrix containing link object at position s, d
        cls.indices = indices    # Mask on matrix links
        cls.ordered_indices = ordered_indices    # Array of tuples (ordered indices)
        
        return links, indices, ordered_indices
        

    @classmethod
    def path_resources(cls, path, sl):    # To check for the available time slots, following the continuity constraint
        available_tdcs = [True] * cls.n_ts["tdc"]    # For traditional data channel slots
        available_ts = [True] * cls.n_ts[sl]    # Creating a base boolean array of the size corresponding to the specific CR's sl

        for s, d in zip(path, path[1:]):    # Taking consecutive pairs of nodes and selecting the particular channel
            if sl == "high":
                band = cls.links[s, d].lambda_q1
            elif sl == "medium":
                band = cls.links[s, d].lambda_q2
            else:
                band = cls.links[s, d].lambda_q3

            # Checking for continuity constraints in ts of quantum channel
            available_ts = [a and b for a, b in zip(available_ts, band)]
            available_tdcs = [a and b for a, b in zip(available_tdcs, links[s, d].lambda_tdc)]
            
        return available_ts, available_tdcs


    @classmethod
    def ASLC(cls, ch, path, aslc, beta_1 = 1, beta_2 = 0.5):
        
        NT = []    # equivalent to dict channel_ts. starts from index 0 : for high priority
        OT = []    # Number of ts occupied along the ENTIRE path
        for i in range(1, 4):
            nt = cls.channel_ts[i]    # Total time slot in channel/n-th wavelength
            NT.append(nt)
                
            available_ts, available_tdcs = cls.path_resources(path, cls.priority[i])    # Available ts in the path for a particular wavelength
            ot = nt - np.count_nonzero(available_ts)    # Denotes the number of time-slot continuity occupied along the path for the given sl
            OT.append(ot)
                
        # ch could be 1, 2 or 3
        n = ch - 1    # n could be 0, 1 or 2

        if aslc == "ASSL" and ch != 1:    # ch = 2 or 3. n = 1 or 2
            if OT[n-1] >= beta_2 * NT[n-1]:    # If the occupied resources in the higher priority are greater than a threshold, don't allocate to it
                QW = cls.priority[ch]
            else:
                QW = cls.priority[ch-1]

        elif aslc == "AWSL" and ch != 3:    # ch = 1 or 2. n = 0 or 1
            if OT[n] >= beta_1 * NT[n]:    # If the resources in current priority are more than a threshold, allocate to a lower priority
                QW = cls.priority[ch+1]
            else:
                QW = cls.priority[ch]

        else:    # For SSL and cases in ASSL but high priority, or AWSL but low priority
            QW = cls.priority[ch]
         
        return QW
            
        
    @classmethod
    def FF(cls, path, cr, aslc = "ssl"):
        # First check for availability of resources

        cr.sl = cls.ASLC(cls.channel[cr.sl], path, aslc)
        sl = cr.sl
        available_ts, available_tdcs = cls.path_resources(path, sl)    # List of available time slots in the channel along the entire path

        if cr.tk > len(available_ts):    # ts is 1 in our case by default. But in case we change it, this line will be useful
            raise ValueError("CR duration cannot be longer than the available time-slots in a channel")
            # Can also block it instead
         
        if np.count_nonzero(available_ts) == 0 or np.count_nonzero(available_tdcs) == 0:    # A pre-condition to check if available slots are present
            cr.update_status("blocked")
            return False

        # Some issue here
        qscs = np.nonzero(available_ts)[0][0]    # This function will give the index of 1st non-zero/True element in the list
        tdcs = np.nonzero(available_tdcs)[0][0]    # At present, assuming only one tdcs is needed
        
        cipher(cr)    # Better to implement in the QKRA file

    @classmethod
    def cipher(cls, cr) :

        path = cr.path
        
        for s, d in zip(path, path[1:]):    # A loop to update all the links in the path 
            link = links[s, d]

            key = link.bb84()    # Incomplete
            link.tdc(key, cr.text)    # Incomplete
            
            link.update_link(cls.channel["tdc"], tdcs)
            link.update_link(cls.channel[sl], qscs)    # Updating the i_th ts in channel for sl to False

        allocated_resources = [cls.channel[sl], qscs]
        cr.update_status("allocated", allocated_resources, path)
        
        return True

        
    @classmethod
    def TUR(cls, channel = 'total'):    # Returns the total TUR, if no 2nd argument given    
        # Only for quantum channel
        util_ts = [0, 0, 0, 0]
        tur = util_ts
        
        total_network_ts = len(cls.ordered_indices) * cls.total_ts    # = 1260
        
        for nodes in cls.ordered_indices :
            link = cls.links[nodes]
            util_ts += link.occupied_ts

        tur[0] = util_ts[0]/total_network_ts
        for i in range(1, 4):
            tur[i] = util_ts[i]/(len(cls.ordered_indices)*cls.channel_ts[i])
        
        print("The time-slot utilization ratio(TUR) is : ", tur)
        
        if channel == 'all':
            return tur

        else: 
            return tur[0]       
            
    
    @classmethod
    def display_all_links(cls, wl_info = False):
        for nodes in cls.ordered_indices :
            cls.links[nodes].display_info(wl_info)


In [1]:
# Example of NSFNET 